### Fine-Tune Flan T5

- Joel Stremmel
- 04-17-23

##### About

Fine-Tune Flan T5 on the formatted data using K-Fold Cross-Validation and save the scores.

https://www.philschmid.de/fine-tune-flan-t5

##### Imports

In [ ]:
import os
import re
import glob
import pickle
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

##### Set Parameters

In [ ]:
max_seq_len = 1024
batch_size = 1
accumulation_steps = 32
lr = 0.00002
weight_decay = 0.01
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_epsilon = 0.00000001
warmup_steps = 2
logging_steps = 1
num_workers = 8
seed = 44
epochs = 5
fp16 = False
colab = False
gradient_checkpointing = False
add_summaries = False
input_dir = './data' # "/content/drive/MyDrive/data/"
model_output_dir = "model_output"
results_dir = './results/' #"/content/drive/MyDrive/results/"
model_key = "flan_t5_small" 
lm_path = "google/flan-t5-small"

##### Optionally Connect to Google Drive

In [ ]:
if colab:

    from google.colab import drive
    drive.mount('/content/drive')

##### Disable Tokenizer Parallelism
This is mostly to avoid warnings.

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

##### Load Formatted Data

In [ ]:
if add_summaries:
    
    with open('data/Xwsum_folds.pkl', 'rb') as f:
        X_folds = pickle.load(f)

else:
    
    with open('data/X_folds.pkl', 'rb') as f:
        X_folds = pickle.load(f)

with open('data/y_folds.pkl', 'rb') as f:
    y_folds = pickle.load(f)

##### Check Data Shape

In [ ]:
assert len(X_folds) == len(y_folds), "Expected the same number of folds in X and y."
X = list(X_folds.values())
y = list(y_folds.values())

##### Check Target Prevalence

In [ ]:
print(f"Target prevalance: {np.mean(np.concatenate(y))}.")

##### Check that GPU is Available

In [ ]:
assert torch.cuda.is_available(), "Run this script on a GPU."
print(torch.__version__)

##### Tokenize Text and Fit Model to Each Fold

In [ ]:
y_probs, y_trues = [], []
for i in range(len(X)):
    
    print(f"Fitting model using fold {i} as out of fold data.")
    
    # Identify train folds and shuffle samples
    X_train, y_train = np.concatenate(X[0:i] + X[i+1:], axis=0), np.concatenate(y[0:i] + y[i+1:], axis=0)
    indices = np.arange(len(y_train))
    np.random.shuffle(indices)
    X_train, y_train = X_train[indices], y_train[indices]
    
    # Identify test folds
    X_test, y_test = X[i], y[i]
    
    # Format text and label data as HuggingFace dataset
    train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
    test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})
    
    # Load model and tokenizer
    # This will reset the model weights with each new iteration
    tokenizer = AutoTokenizer.from_pretrained(lm_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(lm_path)
    
    def preprocess_function(sample, padding="max_length", output_max_seq_len=20):
    
        # Add prefix to the input for t5
        inputs = ["Binary classification: " + item for item in sample["text"]]

        # tokenize inputs
        model_inputs = tokenizer(
            inputs,
            max_length=max_seq_len,
            padding=padding,
            truncation=True
        )

        # Tokenize targets with the `text_target` keyword argument
        labels = tokenizer(
            text_target=sample["text"],
            max_length=output_max_seq_len,
            padding=padding,
            truncation=True
        )

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length":
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]

        return model_inputs
    
    # Tokenize train dataset
    train_dataset = train_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=["text"],
        batch_size=batch_size
    )
    train_dataset.set_format("pt")
    
    # Tokenize test dataset
    test_dataset = test_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=["text"],
        batch_size=batch_size
    )
    test_dataset.set_format("pt")
    
    # Define training arguments
    training_args= Seq2SeqTrainingArguments(
        output_dir=model_output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=accumulation_steps,
        warmup_steps=warmup_steps,
        logging_steps=logging_steps,
        weight_decay=weight_decay,
        learning_rate=lr,
        seed=seed,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        dataloader_num_workers=num_workers,
        fp16=fp16,
        logging_strategy="steps",
        save_strategy="no",
        lr_scheduler_type='linear',
        optim="adamw_torch",
        run_name=model_key,
        gradient_checkpointing=gradient_checkpointing,
        predict_with_generate=True,
        do_eval=False,
        fp16_full_eval=False,
        sharded_ddp=False,
        load_best_model_at_end=True,
        prediction_loss_only=False,
        disable_tqdm=True,
        logging_dir=None
    )
    
    # Define model training
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset
    )
    
    # Train model
    trainer.train()
    
    # Predict on test dataset
    output = trainer.predict(test_dataset, num_beams=5, max_length=20)
    print(output)
    print(output.predictions)
    print(output.label_ids)
    break
    
#     labels = output.label_ids
#     y_prob = torch.sigmoid(torch.tensor(output.predictions).double()).numpy()[:, 1]

#     # Save scores and labels
#     y_probs.append(y_prob)
#     y_trues.append(labels)

##### Save Model Probabilities on Test Folds and True Labels

In [ ]:
with open(os.path.join(results_dir, f'{model_key}_y_trues.pkl'), 'wb') as f:
    pickle.dump(y_trues, f)

with open(os.path.join(results_dir, f'{model_key}_y_probs.pkl'), 'wb') as f:
    pickle.dump(y_probs, f)